In [23]:
import pandas as pd
import os
from utils import *
from utils import df_from_filings

In [24]:
from textblob import TextBlob

In [25]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

In [26]:

# df = pd.DataFrame(data, columns=['cik', 'report_type', 'report_identity', 'file', 'text'])


In [27]:
df = df_from_filings()
df = df.head(100)
df.shape

(100, 5)

In [28]:
%%time
vader_untrained_polarity = df.apply(lambda row: vader.polarity_scores(row['text']), axis=1)


CPU times: user 1min 24s, sys: 1.02 s, total: 1min 25s
Wall time: 1min 26s


In [29]:
%%time
vader_untrained_preprocessed_polarity = df.apply(lambda row: vader.polarity_scores(preprocess(row['text'])), axis=1)


CPU times: user 28.1 s, sys: 287 ms, total: 28.3 s
Wall time: 28.5 s


In [30]:
vader_untrained_polarity = pd.DataFrame(vader_untrained_polarity.values.tolist())
vader_untrained_polarity = vader_untrained_polarity.add_prefix('vader_untrained_polarity_');

vader_untrained_preprocessed_polarity = pd.DataFrame(vader_untrained_preprocessed_polarity.values.tolist())
vader_untrained_preprocessed_polarity = vader_untrained_preprocessed_polarity.add_prefix(
    'vader_untrained_preprocessed_polarity_');

In [31]:
df = pd.concat([df, vader_untrained_polarity, vader_untrained_preprocessed_polarity], axis=1);

In [32]:
#TODO lemma, stem, check if remove frequent is good

In [33]:
%%time
textblob_sentiment_polarity = df.apply(lambda row: TextBlob(row['text']).sentiment.polarity, axis=1)
textblob_sentiment_subjectivity = df.apply(lambda row: TextBlob(row['text']).sentiment.subjectivity, axis=1)

textblob_sentiment = pd.concat([textblob_sentiment_polarity, textblob_sentiment_subjectivity], axis=1)
textblob_sentiment.columns = ['textblob_sentiment_polarity', 'textblob_sentiment_subjectivity']

df = pd.concat([df, textblob_sentiment], axis=1);

CPU times: user 18.3 s, sys: 886 ms, total: 19.2 s
Wall time: 20.4 s


In [34]:
%%time
textblob_preprocessed_sentiment_polarity = df.apply(lambda row: TextBlob(preprocess(row['text'])).sentiment.polarity,
                                                    axis=1)
textblob_preprocessed_sentiment_subjectivity = df.apply(
    lambda row: TextBlob(preprocess(row['text'])).sentiment.subjectivity, axis=1)

textblob_preprocessed_sentiment = pd.concat(
    [textblob_preprocessed_sentiment_polarity, textblob_preprocessed_sentiment_subjectivity], axis=1)
textblob_preprocessed_sentiment.columns = ['textblob_preprocessed_sentiment_polarity',
                                           'textblob_preprocessed_sentiment_subjectivity']

df = pd.concat([df, textblob_preprocessed_sentiment], axis=1);

CPU times: user 10.8 s, sys: 121 ms, total: 10.9 s
Wall time: 11 s


In [35]:
df.describe()


,vader_untrained_polarity_neg,vader_untrained_polarity_neu,vader_untrained_polarity_pos,vader_untrained_polarity_compound,vader_untrained_preprocessed_polarity_neg,vader_untrained_preprocessed_polarity_neu,vader_untrained_preprocessed_polarity_pos,vader_untrained_preprocessed_polarity_compound,textblob_sentiment_polarity,textblob_sentiment_subjectivity,textblob_preprocessed_sentiment_polarity,textblob_preprocessed_sentiment_subjectivity
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.031210,0.811440,0.157260,0.999998,0.050900,0.771100,0.177960,0.999973,0.059390,0.360714,0.063683,0.373841
std,0.007296,0.023794,0.021942,0.000014,0.013506,0.019027,0.019418,0.000080,0.019802,0.026531,0.023524,0.032236
min,0.015000,0.762000,0.103000,0.999900,0.021000,0.730000,0.136000,0.999400,0.020335,0.306712,0.021328,0.267308
25%,0.028000,0.792000,0.141000,1.000000,0.043000,0.756750,0.162000,1.000000,0.042871,0.335443,0.048572,0.356683
50%,0.030000,0.813000,0.157500,1.000000,0.053500,0.772000,0.176500,1.000000,0.061456,0.357999,0.058710,0.374307
75%,0.034000,0.831000,0.177000,1.000000,0.058000,0.784000,0.191000,1.000000,0.075336,0.381375,0.085782,0.395459
max,0.068000,0.869000,0.210000,1.000000,0.095000,0.819000,0.224000,1.000000,0.102249,0.441901,0.113533,0.437323


In [ ]:
import flair

flair_sentiment = flair.models.TextClassifier.load('en-sentiment')


In [ ]:
%%time
def get_flair_sentiment(row):
    s = flair.data.Sentence(row['text'])
    flair_sentiment.predict(s)
    return s.tag, s.score

df_flair_sentiment = df.apply(lambda row: get_flair_sentiment(row)  , axis=1, result_type='expand')


df_flair_sentiment.columns = ['flair_sentiment_tag', 'flair_sentiment_score']

df = pd.concat([df, df_flair_sentiment], axis=1);

In [ ]:
df.shape

In [ ]:
'pos', df.vader_untrained_polarity_pos.mean(), df.vader_untrained_preprocessed_polarity_pos.mean(), 'neg', df.vader_untrained_polarity_neg.mean(), df.vader_untrained_preprocessed_polarity_neg.mean(), 'neu', df.vader_untrained_polarity_neu.mean(), df.vader_untrained_preprocessed_polarity_neu.mean()

In [ ]:
txt = """The Board noted that the Fund outperformed its benchmark and ranked in the first quartile of its Performance Peer Group for the quarter-to-date"""

In [22]:
df

,cik,report_type,report_identity,file,text,vader_untrained_polarity_neg,vader_untrained_polarity_neu,vader_untrained_polarity_pos,vader_untrained_polarity_compound,vader_untrained_preprocessed_polarity_neg,vader_untrained_preprocessed_polarity_neu,vader_untrained_preprocessed_polarity_pos,vader_untrained_preprocessed_polarity_compound,textblob_sentiment_polarity,textblob_sentiment_subjectivity,textblob_preprocessed_sentiment_polarity,textblob_preprocessed_sentiment_subjectivity
0,0001566196,N-CSR,0001193125-13-485974,full-submission.txt,UNITED STATES SECURITIES AND EXCHANGE COMMISS...,0.018,0.836,0.146,1.0,0.021,0.788,0.191,0.9999,0.063043,0.334070,0.081047,0.304610
1,0001566196,N-CSR,0001193125-14-452923,full-submission.txt,UNITED STATES SECURITIES AND EXCHANGE COMMISS...,0.020,0.832,0.148,1.0,0.023,0.781,0.196,0.9999,0.068767,0.338673,0.090162,0.314530
2,0001566196,N-CSR,0001193125-16-805370,full-submission.txt,UNITED STATES SECURITIES AND EXCHANGE COMMISS...,0.022,0.842,0.137,1.0,0.028,0.816,0.156,0.9999,0.042880,0.306712,0.048577,0.267308
3,0001566196,N-CSR,0001193125-15-414451,full-submission.txt,UNITED STATES SECURITIES AND EXCHANGE COMMISS...,0.022,0.844,0.134,1.0,0.023,0.794,0.183,0.9999,0.045747,0.312775,0.056861,0.286745
4,0001566196,N-CSR,0001193125-17-380306,full-submission.txt,UNITED STATES SECURITIES AND EXCHANGE COMMISS...,0.029,0.830,0.141,1.0,0.041,0.796,0.163,0.9999,0.049266,0.326277,0.063275,0.335909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0001568194,N-CSRS,0001193125-17-271745,full-submission.txt,Table of Contents UNITED STATES SECURITIES AN...,0.028,0.784,0.188,1.0,0.058,0.754,0.187,1.0000,0.041375,0.341899,0.028375,0.323919
96,0001568292,N-CSR,0001193125-15-399658,full-submission.txt,Table of Contents UNITED STATES SECURITIES AN...,0.039,0.796,0.165,1.0,0.056,0.749,0.195,1.0000,0.056261,0.376671,0.058706,0.399615
97,0001568292,N-CSR,0001193125-17-363468,full-submission.txt,Table of Contents UNITED STATES SECURITIES AN...,0.036,0.803,0.161,1.0,0.054,0.735,0.211,1.0000,0.056555,0.374013,0.056633,0.395357
98,0001568292,N-CSR,0001193125-19-307865,full-submission.txt,Table of Contents UNITED STATES SECURITIES AN...,0.038,0.794,0.168,1.0,0.059,0.737,0.204,1.0000,0.051855,0.371799,0.058359,0.404039
